In [1]:
import numpy as np
import iris
import matplotlib.pyplot as plt
import matplotlib.cm as mpl_cm
import matplotlib
import matplotlib.ticker as ticker 
from matplotlib import cm
from matplotlib.colors import ListedColormap

from pyproj import Proj
from shapely.geometry import shape

import warnings
warnings.filterwarnings("ignore") ## To ignore UserWarning about collapsing a non-contagious variable

In [2]:
def CheckForLess(list1, val):
 
    # traverse in the list
    for x in list1:
 
        # compare with all the
        # values with value
        if val <= x:
            return False
    return True

In [3]:
def getArea(coords):
    c = {"type": "Polygon",
    "coordinates": [[ (coords[3], coords[1]), (coords[3], coords[0]),
                      (coords[2], coords[0]), (coords[2], coords[1]) ]]}
    lon, lat = zip(*c['coordinates'][0])
    random_string = "+proj=aea +lat_1=" + str(coords[0]) + " +lat_2=" + str(coords[1]) + " +lon_0=" + str((coords[2]+coords[3])/2) ##+ " +lat_0=" + str((coords[0]+coords[1])/2) 
    pro = Proj(random_string)
    x, y = pro(lon, lat)
    poly = {"type": "Polygon", "coordinates": [zip(x, y)]}
    return shape(poly).area

In [4]:
path = r"E:\Datasets_Geoengineering\RCP45_variables/"

T_mean_RCP= iris.load(path + "TSA_RCP45.all.nc")[0][:,82:,:]
T_mean_RCP_data= iris.load(path + "TSA_RCP45.all.nc")[0][:,82:,:].data
TNn_RCP = iris.load(path + "TREFMNAV_RCP45.all.nc")[0][:,82:,:]

Tmean_SAI_01 = iris.load(r"E:\Datasets_Geoengineering\JS_files_new_sel_var\TS_01_sel_var_SAI_cam.all.nc")[0][(-21)*12:,82:,:]
Tmean_SAI_02 = iris.load(r"E:\Datasets_Geoengineering\JS_files_new_sel_var\TS_02_sel_var_SAI_cam.all.nc")[0][(-21)*12:,82:,:]
Tmean_SAI_03 = iris.load(r"E:\Datasets_Geoengineering\JS_files_new_sel_var\TS_03_sel_var_SAI_cam.all.nc")[0][(-21)*12:,82:,:]

Tmean_SAI_01_data = iris.load(r"E:\Datasets_Geoengineering\JS_files_new_sel_var\TS_01_sel_var_SAI_cam.all.nc")[0][(-21)*12:,82:,:].data
Tmean_SAI_02_data = iris.load(r"E:\Datasets_Geoengineering\JS_files_new_sel_var\TS_02_sel_var_SAI_cam.all.nc")[0][(-21)*12:,82:,:].data
Tmean_SAI_03_data = iris.load(r"E:\Datasets_Geoengineering\JS_files_new_sel_var\TS_03_sel_var_SAI_cam.all.nc")[0][(-21)*12:,82:,:].data

TNn_SAI_01_data = iris.load(r"E:\Datasets_Geoengineering\JS_files_new_sel_var\TREFMNAV_01_sel_var_SAI.all.nc")[0][(-21)*12:,82:,:].data
TNn_SAI_02_data = iris.load(r"E:\Datasets_Geoengineering\JS_files_new_sel_var\TREFMNAV_02_sel_var_SAI.all.nc")[0][(-21)*12:,82:,:].data
TNn_SAI_03_data = iris.load(r"E:\Datasets_Geoengineering\JS_files_new_sel_var\TREFMNAV_03_sel_var_SAI.all.nc")[0][(-21)*12:,82:,:].data

landmask = iris.load(path + "TSA_RCP45.all.nc")[0][:,82:,:]

In [5]:
len(TNn_SAI_01_data)

252

In [6]:
def Perm_area(variable, variable_data):
    time = 0
    area_list = []
    TNn_list = []


    while time in range(0,len(variable_data)-2*12):
        area_tot = 0
        for lat in range(0,14):
            for lon in range(0,144):
                T_mean = []
                if landmask.data.mask[0,lat,lon] == False:
                    for i in range(0,24):
                        T_mean.append(variable_data[time+i,lat,lon])
                        #Tmean of 2 years needs to be continuously below zero
                    if CheckForLess(T_mean, 273.15) == True:
                        coords = []
                        coords.append(variable[0,lat:lat+2,lon:lon+2].coord("latitude").points[0])
                        coords.append(variable[0,lat:lat+2,lon:lon+2].coord("latitude").points[1])
                        coords.append(variable[0,lat:lat+2,lon:lon+2].coord("longitude").points[0])
                        coords.append(variable[0,lat:lat+2,lon:lon+2].coord("longitude").points[1])
                        area = getArea(coords) / 1000 / 1000 # in km2
                        area_tot = area_tot + area

        area_list.append(area_tot)
        time = time + 12
    
    return(area_list)


In [7]:
RCP_area = Perm_area(T_mean_RCP, T_mean_RCP_data)
SAI1_area = (Perm_area(Tmean_SAI_01, Tmean_SAI_01_data))
SAI2_area =Perm_area(Tmean_SAI_02, Tmean_SAI_02_data) 
SAI3_area =Perm_area(Tmean_SAI_03, Tmean_SAI_03_data) 

In [8]:
RCP_area_mean = np.mean(RCP_area[-21*12-2*12:])
SAI1_area_mean = np.mean(SAI1_area[-21*12-2*12:])
SAI2_area_mean = np.mean(SAI2_area[-21*12-2*12:])
SAI3_area_mean = np.mean(SAI3_area[-21*12-2*12:])

In [9]:
print("RCP permafrost area:", RCP_area_mean)
print("SAI permafrost area:", (SAI1_area_mean + SAI2_area_mean + SAI3_area_mean)/3)

RCP permafrost area: 2120064.310563616
SAI permafrost area: 1955148.8991622524


In [11]:
SAI_area = (SAI1_area_mean + SAI2_area_mean + SAI3_area_mean)/3
print("percentage smaller:", 100 - SAI_area / RCP_area_mean * 100)

percentage smaller: 7.77879286867109
